In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

In [115]:
train = pd.read_csv('../data/v3/training_seg2.csv')
test = pd.read_csv('../data/v3/test_seg2.csv')

In [116]:
train.shape, test.shape

((12474, 14), (93, 14))

In [117]:
train.head()

application_date  segment  case_count  day  year  month      Holiday Type  \
0       2017-04-01        2         0.0    1  2017      4          NaN  NaN   
1       2017-04-02        2         0.0    2  2017      4          NaN  NaN   
2       2017-04-03        2         0.0    3  2017      4          NaN  NaN   
3       2017-04-04        2         0.0    4  2017      4  Rama Navami    G   
4       2017-04-05        2         0.0    5  2017      4          NaN  NaN   

    cos_day   sin_day  cos_mon   sin_mon  day_of_week  week_num  
0  0.978148  0.207912     -0.5  0.866025            5        13  
1  0.913545  0.406737     -0.5  0.866025            6        13  
2  0.809017  0.587785     -0.5  0.866025            0        14  
3  0.669131  0.743145     -0.5  0.866025            1        14  
4  0.500000  0.866025     -0.5  0.866025            2        14

In [118]:
train['log_case'] = np.log1p(train['case_count'])

# Feature Engineering

In [119]:
a1 = [0] + [i for i in train['log_case'].iloc[:-1].tolist()]
a2 = [0] + [0] + [i for i in train['log_case'].iloc[:-2].tolist()]
a3 = [0] + [0] + [0] + [i for i in train['log_case'].iloc[:-3].tolist()]
a4 = [0] + [0] + [0] + [0] + [i for i in train['log_case'].iloc[:-4].tolist()]

In [120]:
train['case_count_prev'] = a1
train['case_count_prev_2'] = a2
train['case_count_prev_3'] = a3
train['case_count_prev_4'] = a4

# Modelling

In [121]:
train.Type.fillna(value="Nope", inplace=True)
test.Type.fillna(value="Nope", inplace=True)

In [122]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [123]:
la = LabelEncoder()
on = OneHotEncoder()
on2 = OneHotEncoder()

In [124]:
train['Type_Mat'] = la.fit_transform(train['Type'])

In [125]:
type_mat = on.fit_transform(train['Type_Mat'].values.reshape(-1,1))
day_mat = on2.fit_transform(train['day_of_week'].values.reshape(-1,1))
type_mat = type_mat.todense()
day_mat = day_mat.todense()
mat = np.append(type_mat, day_mat, axis=1)

C:\Users\EW\AppData\Roaming\Python\Python36\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\EW\AppData\Roaming\Python\Python36\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "cate

In [207]:
mat = np.append(type_mat, day_mat, axis=1)
mat = np.append(mat, train['day'].values.reshape(-1, 1), axis=1)
mat = np.append(mat, train['week_num'].values.reshape(-1, 1), axis=1)
mat = np.append(mat, train['month'].values.reshape(-1, 1), axis=1)
mat = np.append(mat, train['year'].values.reshape(-1, 1), axis=1)
mat = np.append(mat, train['sin_mon'].values.reshape(-1, 1), axis=1)
mat = np.append(mat, train['cos_mon'].values.reshape(-1, 1), axis=1)
mat = np.append(mat, train['sin_day'].values.reshape(-1, 1), axis=1)
mat = np.append(mat, train['cos_day'].values.reshape(-1, 1), axis=1)
mat = np.append(mat, np.log1p(train['case_count_prev'].values.reshape(-1, 1)), axis=1)
mat = np.append(mat, np.log1p(train['case_count_prev_2'].values.reshape(-1, 1)), axis=1)
mat = np.append(mat, np.log1p(train['case_count_prev_3'].values.reshape(-1, 1)), axis=1)
mat = np.append(mat, np.log1p(train['case_count_prev_4'].values.reshape(-1, 1)), axis=1)

In [208]:
mat.shape

(12474, 26)

In [209]:
from sklearn.linear_model import LinearRegression, HuberRegressor, Lasso
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import lightgbm as lgb
import xgboost
from xgboost import XGBRegressor

In [346]:
X_train, X_test, y_train, y_test = train_test_split(mat,train['log_case'], test_size=0.3, random_state=10)

In [354]:
rf = RandomForestRegressor(n_estimators=100)
ls = Lasso()
hu = HuberRegressor()
LGB = lgb.LGBMRegressor()
xgb = XGBRegressor()

In [355]:
LGB.fit(X_train, y_train)
ls.fit(X_train, y_train)
hu.fit(X_train, y_train)
rf.fit(X_train, y_train)
xgb.fit(X_train, y_train)

e:\python36\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, importance_type='gain',
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
       nthread=None, objective='reg:linear', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1)

In [356]:
lgb_pred = abs(np.round(np.expm1(LGB.predict(X_test))))
xgb_pred = abs(np.round(np.expm1(xgb.predict(X_test))))
rf_pred = abs(np.round(np.expm1(rf.predict(X_test))))
ls_pred = abs(np.round(np.expm1(ls.predict(X_test))))
hu_pred = abs(np.round(np.expm1(hu.predict(X_test))))

In [357]:
[mean_absolute_error(np.expm1(y_test), lgb_pred) * (100/len(y_test)),
mean_absolute_error(np.expm1(y_test), xgb_pred) * (100/len(y_test)),
mean_absolute_error(np.expm1(y_test), rf_pred) * (100/len(y_test)),
mean_absolute_error(np.expm1(y_test), ls_pred) * (100/len(y_test)),
mean_absolute_error(np.expm1(y_test), hu_pred) * (100/len(y_test)),
]

[3.1504315224022412,
 3.3648633206065157,
 3.309510195146355,
 9.694784079627416,
 6.020949676906912]

In [285]:
xtest = train.iloc[y_test.index].reset_index(drop=True)
xtest['log_case'] = y_test.values
xtest['lgb_pred'] = lgb_pred
xtest['xgb_pred'] = xgb_pred
xtest['hu_pred'] = hu_pred
xtest['ls_pred'] = ls_pred
xtest['rf_pred'] = rf_pred

In [286]:
xtest[['log_case', 'case_count_prev', 'case_count_prev_2',
       'case_count_prev_3', 'case_count_prev_4', 
       'hu_pred', 'ls_pred', 'rf_pred', 'lgb_pred', 'xgb_pred', 'case_count']].to_csv('stack_10.csv')

# Loading Stacked Data

In [289]:
pk = []
for i in range(1, 11):
    pk.append(pd.read_csv(f'stack_{i}.csv'))

In [290]:
df = pd.concat(pk, ignore_index=True, sort=False)
df.drop(columns='Unnamed: 0', inplace=True)

In [308]:
df.head()

log_case  case_count_prev  case_count_prev_2  case_count_prev_3  \
0  7.053586         7.122060           6.476972           7.649693   
1  0.000000         0.000000           0.000000           0.000000   
2  7.390799         6.608001           5.552960           4.736198   
3  0.000000         0.000000           0.000000           0.000000   
4  0.000000         0.000000           0.000000           0.000000   

   case_count_prev_4  hu_pred  ls_pred  rf_pred  lgb_pred  xgb_pred  \
0           7.769801    719.0    272.0   1022.0     695.0    1213.0   
1           0.000000      0.0      8.0      0.0       0.0       0.0   
2           5.826000    436.0    245.0    794.0     804.0     753.0   
3           0.000000      0.0      8.0      0.0       0.0       0.0   
4           0.000000      0.0      8.0      0.0       0.0       0.0   

   case_count  
0      1156.0  
1         0.0  
2      1620.0  
3         0.0  
4         0.0

In [309]:
df.corr()

log_case  case_count_prev  case_count_prev_2  \
log_case           1.000000         0.966506           0.942660   
case_count_prev    0.966506         1.000000           0.966557   
case_count_prev_2  0.942660         0.966557           1.000000   
case_count_prev_3  0.923539         0.943995           0.966670   
case_count_prev_4  0.906510         0.923277           0.941795   
hu_pred            0.831052         0.842238           0.837757   
ls_pred            0.952294         0.990789           0.953331   
rf_pred            0.696113         0.701552           0.669644   
lgb_pred           0.702279         0.704017           0.671620   
xgb_pred           0.700509         0.709816           0.675691   
case_count         0.705038         0.668321           0.640679   

                   case_count_prev_3  case_count_prev_4   hu_pred   ls_pred  \
log_case                    0.923539           0.906510  0.831052  0.952294   
case_count_prev             0.943995           0.923277  0.842238  0.990789   
case_count_prev_2           0.966670           0.941795  0.837757  0.953331   
case_count_prev_3           1.000000           0.965956  0.822358  0.926402   
case_count_prev_4           0.965956           1.000000  0.799123  0.902154   
hu_pred                     0.822358           0.799123  1.000000  0.880083   
ls_pred                     0.926402           0.902154  0.880083  1.000000   
rf_pred                     0.639001           0.612696  0.865385  0.771862   
lgb_pred                    0.642103           0.616505  0.867109  0.773907   
xgb_pred                    0.645763           0.619569  0.877066  0.780921   
case_count                  0.613039           0.590966  0.818458  0.732361   

                    rf_pred  lgb_pred  xgb_pred  case_count  
log_case           0.696113  0.702279  0.700509    0.705038  
case_count_prev    0.701552  0.704017  0.709816    0.668321  
case_count_prev_2  0.669644  0.671620  0.675691    0.640679  
case_count_prev_3  0.639001  0.642103  0.645763    0.613039  
case_count_prev_4  0.612696  0.616505  0.619569    0.590966  
hu_pred            0.865385  0.867109  0.877066    0.818458  
ls_pred            0.771862  0.773907  0.780921    0.732361  
rf_pred            1.000000  0.986819  0.984302    0.923310  
lgb_pred           0.986819  1.000000  0.989106    0.931282  
xgb_pred           0.984302  0.989106  1.000000    0.921720  
case_count         0.923310  0.931282  0.921720    1.000000

In [336]:
training = df[['log_case', 'case_count_prev', 'case_count_prev_2', 'case_count_prev_3',
               'case_count_prev_4', 'hu_pred', 'ls_pred', 'rf_pred', 'lgb_pred',
               'xgb_pred']]
label = df['case_count']

In [337]:
X_train, X_test, y_train, y_test = train_test_split(training, label, test_size=0.3, random_state=42)

In [338]:
rf = RandomForestRegressor(n_estimators=100)

In [339]:
rf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [340]:
pred = rf.predict(X_test)

In [341]:
xtest = X_test.loc[y_test.index].reset_index(drop=True)

In [344]:
mean_absolute_error(y_test, pred) * (100/len(y_test))

0.00035273094175312963

# Testing

In [358]:
test['Type_Mat'] = la.fit_transform(test['Type'])
type_mat_test = on.transform(test['Type_Mat'].values.reshape(-1,1))
day_mat_test = on2.transform(test['day_of_week'].values.reshape(-1,1))
type_mat_test = type_mat_test.todense()
day_mat_test = day_mat_test.todense()
mat_test = np.append(type_mat_test, day_mat_test, axis=1)

In [359]:
mat_test = np.append(mat_test, test['day'].values.reshape(-1, 1), axis=1)
mat_test = np.append(mat_test, test['week_num'].values.reshape(-1, 1), axis=1)
mat_test = np.append(mat_test, test['month'].values.reshape(-1, 1), axis=1)
mat_test = np.append(mat_test, test['year'].values.reshape(-1, 1), axis=1)
mat_test = np.append(mat_test, test['sin_mon'].values.reshape(-1, 1), axis=1)
mat_test = np.append(mat_test, test['cos_mon'].values.reshape(-1, 1), axis=1)
mat_test = np.append(mat_test, test['sin_day'].values.reshape(-1, 1), axis=1)
mat_test = np.append(mat_test, test['cos_day'].values.reshape(-1, 1), axis=1)

In [360]:
mat.shape, mat_test.shape

((12474, 26), (93, 22))

In [379]:
def func(model):
    a = []
    for i in range(len(mat_test)):
        if i ==0:
            k = np.append(np.array(mat_test[i]), np.log1p(np.array([[1480., 1028., 1946., 1984.]])), axis=1)
    #         a.append(np.round(np.expm1(rf.predict(k))))
        if i==1:
            k = np.append(np.array(mat_test[i]), np.array([[a[0],0,0,0]]), axis=1)
    #         a.append(np.round(np.expm1(rf.predict(k))))
        if i==2:
            k = np.append(np.array(mat_test[i]), np.array([[a[1],a[0],0,0]]), axis=1)
    #         a.append(np.round(np.expm1(rf.predict(k))))
        if i==3:
            k = np.append(np.array(mat_test[i]), np.array([[a[2],a[1],a[0],0]]), axis=1)
    #         a.append(np.round(np.expm1(rf.predict(k))))
        if i>3:
            k = np.append(np.array(mat_test[i]), np.array([[a[i-1],a[i-2],a[i-3],a[i-4]]]), axis=1)

        a.append(np.round(np.expm1(model.predict(k)))[0])
    return a

In [385]:
a_rf = func(rf)
# a_ls = func(ls)
# a_hu = func(hu)
a_LGB = func(LGB)
a_xgb = func(xgb)

In [382]:
test['case_count'] = a
test['log_case'] = np.log1p(a)

In [368]:
# test[['id','application_date','segment','case_count']].to_csv('pred_prev_seg_2.csv', index=False)

In [373]:
np.log1p([1480., 1028., 1946., 1984.])

array([7.30047281, 6.93634274, 7.57404501, 7.59337419])

In [374]:
a1 = [7.30047281] + [i for i in test['log_case'].iloc[:-1].tolist()]
a2 = [6.93634274] + [7.30047281] + [i for i in test['log_case'].iloc[:-2].tolist()]
a3 = [7.57404501] + [6.93634274] + [7.30047281] + [i for i in test['log_case'].iloc[:-3].tolist()]
a4 = [7.59337419] + [7.57404501] + [6.93634274] + [7.30047281] + [i for i in test['log_case'].iloc[:-4].tolist()]

In [375]:
test['case_count_prev'] = a1
test['case_count_prev_2'] = a2
test['case_count_prev_3'] = a3
test['case_count_prev_4'] = a4

In [376]:
test

id application_date  segment  day  year  month  \
0    88       2019-07-24        2   24  2019      7   
1    89       2019-07-25        2   25  2019      7   
2    90       2019-07-26        2   26  2019      7   
3    91       2019-07-27        2   27  2019      7   
4    92       2019-07-28        2   28  2019      7   
5    93       2019-07-29        2   29  2019      7   
6    94       2019-07-30        2   30  2019      7   
7    95       2019-07-31        2   31  2019      7   
8    96       2019-08-01        2    1  2019      8   
9    97       2019-08-02        2    2  2019      8   
10   98       2019-08-03        2    3  2019      8   
11   99       2019-08-04        2    4  2019      8   
12  100       2019-08-05        2    5  2019      8   
13  101       2019-08-06        2    6  2019      8   
14  102       2019-08-07        2    7  2019      8   
15  103       2019-08-08        2    8  2019      8   
16  104       2019-08-09        2    9  2019      8   
17  105       2019-08-10        2   10  2019      8   
18  106       2019-08-11        2   11  2019      8   
19  107       2019-08-12        2   12  2019      8   
20  108       2019-08-13        2   13  2019      8   
21  109       2019-08-14        2   14  2019      8   
22  110       2019-08-15        2   15  2019      8   
23  111       2019-08-16        2   16  2019      8   
24  112       2019-08-17        2   17  2019      8   
25  113       2019-08-18        2   18  2019      8   
26  114       2019-08-19        2   19  2019      8   
27  115       2019-08-20        2   20  2019      8   
28  116       2019-08-21        2   21  2019      8   
29  117       2019-08-22        2   22  2019      8   
..  ...              ...      ...  ...   ...    ...   
63  151       2019-09-25        2   25  2019      9   
64  152       2019-09-26        2   26  2019      9   
65  153       2019-09-27        2   27  2019      9   
66  154       2019-09-28        2   28  2019      9   
67  155       2019-09-29        2   29  2019      9   
68  156       2019-09-30        2   30  2019      9   
69  157       2019-10-01        2    1  2019     10   
70  158       2019-10-02        2    2  2019     10   
71  159       2019-10-03        2    3  2019     10   
72  160       2019-10-04        2    4  2019     10   
73  161       2019-10-05        2    5  2019     10   
74  162       2019-10-06        2    6  2019     10   
75  163       2019-10-07        2    7  2019     10   
76  164       2019-10-08        2    8  2019     10   
77  165       2019-10-09        2    9  2019     10   
78  166       2019-10-10        2   10  2019     10   
79  167       2019-10-11        2   11  2019     10   
80  168       2019-10-12        2   12  2019     10   
81  169       2019-10-13        2   13  2019     10   
82  170       2019-10-14        2   14  2019     10   
83  171       2019-10-15        2   15  2019     10   
84  172       2019-10-16        2   16  2019     10   
85  173       2019-10-17        2   17  2019     10   
86  174       2019-10-18        2   18  2019     10   
87  175       2019-10-19        2   19  2019     10   
88  176       2019-10-20        2   20  2019     10   
89  177       2019-10-21        2   21  2019     10   
90  178       2019-10-22        2   22  2019     10   
91  179       2019-10-23        2   23  2019     10   
92  180       2019-10-24        2   24  2019     10   

                            Holiday  Type       cos_mon   sin_mon  ...  \
0                               NaN  Nope -8.660254e-01 -0.500000  ...   
1                               NaN  Nope -8.660254e-01 -0.500000  ...   
2                               NaN  Nope -8.660254e-01 -0.500000  ...   
3                               NaN  Nope -8.660254e-01 -0.500000  ...   
4                               NaN  Nope -8.660254e-01 -0.500000  ...   
5                               NaN  Nope -8.660254e-01 -0.500000  ...   
6                               NaN  Nope -8.660254e-01 -0.500000  ...   
7 